## COMP90089 Assignment1


### Question 2 
**List the criteria used by clinicians to define Hypotension.**

Hypotension is one of the features of shock. It occurs in the majority of patients with shock. 


Hypotension may be 
* absolute (eg, systolic blood pressure <90 mmHg; mean arterial pressure <65 mmHg), 
* relative (eg, a drop in systolic blood pressure >40 mmHg), 
* orthostatic (>20 mmHg fall in systolic pressure or >10 mmHg fall in diastolic pressure with standing), or 
* profound (eg, vasopressor-dependent). 

Not every patient who has hypotension has shock (eg, chronic hypotension, autonomic dysfunction, vasovagal syncope, peripheral vascular disease)



### Question 3


**List the International Classification of Diseases (ICD) codes used to describe hypotension**

* I95 Hypotension
* I95.0 Idiopathic hypotension
* I95.1 Orthostatic hypotension
* I95.2 Hypotension due to drugs
* I95.8 Other hypotension, Chronic hypotension
* I95.9 Hypotension, unspecified



### Question 4

**ind the concept that would match the ICD-10 code for hypotension. Is there a perfect match? Please describe the potential pros and cons of
mapping between one terminology (ICD codes) and another (SNOMED-CT) when doing a data analysis project that requires linking data from more than one source.**


There is no perfect match. Some codes perfectly match (e.g.28651003 and I95.1 for Orthostatic hypotension), but some do not.
There are 5 Hypotensions under the Hypotension categories in ICD codes but 13 Hypotensions under the Hypotension category in SNOMED-CT. Overall, the ICD code is more general and has a category called **other hypotension** to collect relative rare hypotension like Chronic hypotension. The SNOMED-CT is designed for input into EHRs, and the terms are pretty detailed for reporting. ICD-10 is a classification created for output or reports. Patients may be admitted to a hospital that uses ICD codes and then admitted to another that uses SNOMED-CT code, and vice versa.

For the data linkage task,
Pros
* If there is a perfect match between the ICD code and the SNOMED-CT code, then we can use both codes to find the cohort of patients. Consequently, they can all be captured if the patient's records are in ICD or SNOMED-CT. This will make our cohort more precise.
* As mentioned, the ICD Code is more general, and the SNOMED-CT code is more fine-grained. Introducing the SNOMED-CT to our dataset could be more precisely determine the disease, which would better for us to understand the patients

Cons
* As mentioned, the two codes are in different grained levels. If we record patients only in ICD code or SNOMED-CT code, we may lose some information when we want to transfer the record to another code representation. For example, if one patient has Maternal hypotension syndrome(SNOMED-CT: 88887003), there is no perfect match to the ICD code, we can only use the ICD code "I95.8 Other hypotension" to represent this "Maternal hypotension syndrome". This may cost medical staff cost more time to make the diagnosis.
* Same to ICD code, if the patient is recorded in ICD code with "I95.8
Other hypotension"， then when it transfers to SNOMED-CT, there won't be a match for Other hypotension in SNOMED, which may lead the transfer to fail.

### Question 5
**Find patients that meet this definition of profound hypotension**:

Hypotension that lasted >60 minutes and was corrected after the use of intravenous vasoactive drugs.

We follow those steps to find the target patients:


1.   **Find what the intravenous vasoactive drugs are** 
> We do not have much information about intravenous vasoactive drugs. We extract the information from ***mimiciv_derived.vasoactive_agent***, which list some vasoactive drugs. We assume all the drugs listed in this table are the intravenous vasoactive drugs we are looking for.


2.   **Find the item id for these intravenous vasoactive drugs**
> We have to find the corresponding code for the vasoactive drugs since all information is recorded in Item CODE rather than their name. We use name string  from table ***mimiciv_icu.d_items*** to extract all item code for each Drug (case insensitive)


3.   **Find the patients who use those drugs**
> Then we need to figure out which patient used these drugs. We used the data table from ***ICU module***. The ICU module contains a larger cohort than the hospital module. Thus we may find the cohort more accurate. We extracted data from ***mimiciv_icu.inputevents*** and ***mimiciv_icu.precedureevents***. Since the drug-taking event is the information documented for continuous infusions or intermittent administrations or Procedures documented. It is not patient outputs or recorded in a chart, so we won't consider ***outputevent and chartevent***. We named the cohort table as **drugs table**.



4.   **Decide Blood Pressure measurements to determine Hypotension**
> We use the definition from UpToDate, and there are four criteria: 
> 1.   absolute (eg, systolic blood pressure <90 mmHg; mean arterial pressure <65 mmHg)
>2.   relative (e.g., a drop in systolic blood pressure >40 mmHg)
>3. orthostatic (>20 mmHg fall in systolic pressure or >10 mmHg fall in diastolic pressure withstanding)
>4. profound (e.g., vasopressor-dependent)
> Here, we only consider the **Absolute** and **Relative**. 
Since the Orthostatic happens in a particular position or action, it is a bit hard to observe that. Also, we do not have much information about vasopressor-dependent. Thus the cohort who satisfied absolute criteria will be selected. To judge whether the patient have absolute or relative, we have to use three Blood Pressure measurements: **systolic blood pressure, diastolic blood pressure** and **mean arterial pressure**


5. **Find the item id for these Blood Pressure Measurements**
> We find all labels that contains strings **"systolic", "diastolic" and "arterial blood"** from ***mimiciv_icu.d_items***. We got lots of relevant code, but we decided to choose the following code as our measurements:
>* 220051      Arterial Blood Pressure diastolic
>* 220052      Arterial Blood Pressure mean
>* 220050       Arterial Blood Pressure systolic
>* 220056    Arterial Blood Pressure Alarm - Low
We also consider the 220056 Blood Pressure Alarm-LOW could be useful in detecting Hypotension, so we also keep code 220056.

6. **Find the cohort who had Blood Pressure Measurements using these codes**
> We tried to extract patient data through ***mimiciv_icu.inputevents, mimicv_icu.chartevents*** and ***mimiciv_icu.precedureevents***, but the target data only exist on ***mimicv_icu.chartevents***. Thus, we keep the cohort with code **220051,220052,220050 and 220056** from ***mimicv_icu.chartevents***. We name the table as **bp table**.


7.   **Data cleaning and combining**
> Then, we did some data cleaning steps. We only keep patient identifiers **subject_id, hadm_id and stay_id** and the **time of Drug taking**for the drug table. For bp table, we keep patients identifiers **subject_id, hadm_id and stay_id**, 
the **type of measurement** and the **figure of measurements**. Then, we aim to make a timeline for each patient each stay. For each timeslot, event(s) happened(s). These events could be drug-taking or measuring blood pressure. Thus, we combine the two tables and order them by the patients' identifiers and time.

8. **Label state for each timeslot**
> For each stay patient, we have ordered all events in time. We want to label the state for each timeslot to judge if this patient meets this profound hypotension definition. We have three state labels **(Hypotension, Drug and Normal)**. We first label the state with the Drug, if the patient takes the Drug in this timeslot, then this will be labelled as **Drug**. Then, if the patient has blood pressure measurement, we will use the rule of **Absolute and Relative** to check if the patient is having Hypotension or not. We found the **Relative** needs a drop of Blood Pressure diastolic measurement, but this should happen spontaneously, which is quite challenging for us to catch, so we only keep the **Absolute** rule. If the systolic blood pressure is <90 mmHg and the mean arterial pressure is <65 mmHg, we will label this timeslot as **Hypotension**. Also, if an Arterial Blood Pressure Alarm - Low is recorded in this timeslot, we will also label this stage as **Hypotension**. Otherwise, we will label the rest of the timeslot as **Normal**. We also apply a filter to keep valid patients. The definition of profound Hypotension has three stages( Hypotension last>60min -> Take Drugs -> Blood Pressure Corrected)， which means it requires our patient to have all three labels(Hypotension, Drug and Normal). We apply a filter to keep patients who have all three labels.


9. **Deal with conflict labels. Only keep one label for each timeslot**
> We found the labels in each timeslot may have conflicts. For example, a patient has to mean arterial and systolic blood pressure simultaneously, but only systolic blood pressure measurement indicates Hypotension, and the mean arterial pressure measurement does not. If multiple events happen during this time, we will use the priority of **<Drug, Hypotension, Normal>** to label each timeslot. After that, each timeslot will only have one label to indicate the current stage.


10. **Check if each patient for each stay is valid for check_profound_hypotension**
> We built a function called check_profound_hypotension. We input each patient's data for each stay with a timeslot and the label for each timeslot, and then it will return a Boolean value to show if this patient for this stay is valid for profound Hypotension. As mentioned, profound Hypotension has three stages: **(Hypotension last>60min -> Take Drugs -> Blood Pressure Corrected)**. We iterate each timeslot. If the Hypotension label occurs, then we record the Hypotension start time. We keep doing the iteration and sum up all the Hypotension last time; if the last time is over 60 mins, that means the first stage is satisfied, then we will find the coming drug take label and record the time, and the second stage is done. After that, we will check if there is a Normal label after drug taking. If there is one, then all stages are complete, and we can see this patient in this stay meet the profound_hypotension and stop iteration to save time. Note that we have a constraint for the drug taking: we only see Drug taking as valid if it is taken within 90 mins after the first stage. Otherwise, we don't know if the Drug helps the patient recover or if they recover by themselves. We may need to do a further constraint about the drug effect time since it may need some time for a drug to correct blood pressure, and we do not know if the Drug did help for blood pressure correction. We do not have this actual effect time. Thus we will treat all corrections as the credit of the drug. After that, we build our cohort table, we keep the record of each patients' identifiers and also a True value to indicates they are valid for profound hypotension,. We aggrate all timeslot together so there is no "remaining columns used for filtering" for us to keep. 

11. **Extract patients' ICD Code**
> We also need a column that states whether
Any ICD code for hypotension was added to their list of diagnoses when they were discharged, and we use ***mimiciv_hosp.diagnoses_icd*** table to search all **icd_code** that like **("I95%")** since all hypotension related code starts with **I95**. We name this table the ICD table, which contains the patients' identifiers **hadm_id** with codes like **I95** and their ICD codes.

12. **Link the ICD table and the Cohort Table**
> We use **hadm_id** as the key, iterate each **hadm_id** from the Cohort table and check if this **hadm_id** occurs in the ICD table. It appears, we will label this **hadm_id** as True. Otherwise False. Then this boolean value will be added to our cohort table to get the final cohort table.

The implementation algorithem will be introduced with code chuncks

### Code Implementation

In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
import warnings
warnings.filterwarnings('ignore')

In [2]:
# authenticate
auth.authenticate_user()

In [3]:
# Set up environment variables
project_id = 'clinical-entity-extraction'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

#### 1. **Find what the intravenous vasoactive drugs are**
We first want to find what are the intravenous vasoactive drugs.
From *Mimiciv.deveived* we found a table called vasoactive agent, we extract the column name which are the intravenous vasoactive drugs.

In [4]:
vasoactive_drugs = run_query("""

SELECT * FROM `physionet-data.mimiciv_derived.vasoactive_agent` 
WHERE 1=0
""")
vasoactive_drugs = vasoactive_drugs.columns[3:].tolist()

In [5]:
print("Possible intravenous vasoactive drugs: ",", ".join(x for x in vasoactive_drugs) )

Possible intravenous vasoactive drugs:  dopamine, epinephrine, norepinephrine, phenylephrine, vasopressin, dobutamine, milrinone


#### 2. **Find the item id for these intravenous vasoactive drugs**
we use *ditem* table from ICU module to find the id for each drug

In [6]:
##Find Drug Id:
vasoactive_drugs_df = run_query("""
SELECT itemid, label
FROM `physionet-data.mimiciv_icu.d_items` 
WHERE UPPER(label) LIKE UPPER("dopamine") OR UPPER(label) LIKE UPPER("epinephrine") OR
UPPER(label) LIKE UPPER("norepinephrine") OR UPPER(label) LIKE UPPER("phenylephrine") OR
UPPER(label) LIKE UPPER("vasopressin") OR UPPER(label) LIKE UPPER("dobutamine") OR
UPPER(label) LIKE UPPER("milrinone")

""")
print(vasoactive_drugs_df)


   itemid           label
0  221289     Epinephrine
1  221653      Dobutamine
2  221662        Dopamine
3  221749   Phenylephrine
4  221906  Norepinephrine
5  221986       Milrinone
6  222315     Vasopressin


#### 3. **Find the patients who use those drugs**
Here we only consider ICU module, since ICU contains more data.
Use extracted Drug id to find patients who uses those drugs from ICU module

We tried input, procedure events, but only inputevent contains valid data. Thus we keep input events as our drugs table.


In [7]:
##Module icu:
drugs = run_query("""

SELECT *
FROM `physionet-data.mimiciv_icu.inputevents`  
WHERE itemid in (221289,221653,221662,221749,221906,221986,222315)
ORDER BY subject_id

""")
print(drugs.head())

        subject_id   hadm_id   stay_id           starttime  \
0         10001884  26184834  37510196 2131-01-11 09:14:00   
1         10001884  26184834  37510196 2131-01-11 08:17:00   
2         10001884  26184834  37510196 2131-01-11 04:50:00   
3         10001884  26184834  37510196 2131-01-11 06:54:00   
4         10001884  26184834  37510196 2131-01-11 05:04:00   
...            ...       ...       ...                 ...   
644646    19999840  21033226  38978960 2164-09-15 22:04:00   
644647    19999840  21033226  38978960 2164-09-17 05:44:00   
644648    19999840  21033226  38978960 2164-09-17 00:10:00   
644649    19999840  21033226  38978960 2164-09-16 17:25:00   
644650    19999840  21033226  38978960 2164-09-16 15:51:00   

                   endtime           storetime  itemid     amount amountuom  \
0      2131-01-11 10:01:00 2131-01-11 09:35:00  221662   6.113676        mg   
1      2131-01-11 09:14:00 2131-01-11 08:30:00  221662  22.243373        mg   
2      2131-01-11 

In [10]:
procedure_event_df = run_query("""
SELECT  procev.*,dlab.label
FROM `physionet-data.mimiciv_icu.procedureevents`  AS procev 
INNER JOIN `physionet-data.mimiciv_icu.d_items` AS dlab
ON procev.itemid = dlab.itemid
WHERE procev.itemid in (221289,221653,221662,221749,221906,221986,222315)
ORDER BY subject_id

""")
print(procedure_event_df)

Empty DataFrame
Columns: [subject_id, hadm_id, stay_id, starttime, endtime, storetime, itemid, value, valueuom, location, locationcategory, orderid, linkorderid, ordercategoryname, ordercategorydescription, patientweight, isopenbag, continueinnextdept, statusdescription, ORIGINALAMOUNT, ORIGINALRATE, label]
Index: []

[0 rows x 22 columns]


In [11]:
drugs.head()

,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,amount,amountuom,rate,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10001884,26184834,37510196,2131-01-11 09:14:00,2131-01-11 10:01:00,2131-01-11 09:35:00,221662,6.113676,mg,2.001203,...,Main order parameter,Continuous Med,65.0,250.0,ml,0,0,Paused,292.675964,2.0
1,10001884,26184834,37510196,2131-01-11 08:17:00,2131-01-11 09:14:00,2131-01-11 08:30:00,221662,22.243373,mg,6.003609,...,Main order parameter,Continuous Med,65.0,250.0,ml,0,0,ChangeDose/Rate,314.919342,6.0
2,10001884,26184834,37510196,2131-01-11 04:50:00,2131-01-11 05:04:00,2131-01-11 04:50:00,221662,9.105691,mg,10.006255,...,Main order parameter,Continuous Med,65.0,250.0,ml,0,0,ChangeDose/Rate,400.000000,10.0
3,10001884,26184834,37510196,2131-01-11 06:54:00,2131-01-11 07:40:00,2131-01-11 06:48:00,221662,20.943089,mg,7.004377,...,Main order parameter,Continuous Med,65.0,250.0,ml,0,0,ChangeDose/Rate,355.121948,7.0
4,10001884,26184834,37510196,2131-01-11 05:04:00,2131-01-11 06:54:00,2131-01-11 05:04:00,221662,35.772359,mg,5.003127,...,Main order parameter,Continuous Med,65.0,250.0,ml,0,0,ChangeDose/Rate,390.894318,5.0


#### 4. **Decide Blood Pressure measurements to determine Hypotension**
We use the definition from UpToDate find four criteria: 
**absolute, relative, orthostatic, profound**. Since the Orthostatic happens in a particular position or action, it is a bit hard to observe that. Also, we do not have much information about vasopressor-dependent. Thus the cohort who satisfied absolute criteria will be selected. To judge whether the patient have absolute or relative, we have to use three Blood Pressure measurements: **systolic blood pressure, diastolic blood pressure** and **mean arterial pressure**





#### 5. **Find the item id for these Blood Pressure Measurements**
Here we only consider Absolute and Relative, and **systolic blood pressure, diastolic blood pressure** and **mean arterial pressure** will be used to determine Absolute and Relative. We search the label like ("%systolic%") ("%diastolic%") and ("%arterial blood%") and also ignore case from the table *mimiciv_icu.d_item*. 


Here are lots of labels but we choose:
* 220051      Arterial Blood Pressure diastolic
* 220052      Arterial Blood Pressure mean
* 220050       Arterial Blood Pressure systolic

as the three measurements to represent the blood pressure measure. We also notice the 

* 220056    Arterial Blood Pressure Alarm - Low

might be useful for us to detect Hypotension


In [12]:
blood_pressure_df = run_query("""

SELECT itemid, label
FROM `physionet-data.mimiciv_icu.d_items` 
WHERE UPPER(label) LIKE UPPER("%systolic%") OR UPPER(label) LIKE UPPER("%diastolic%") OR
     UPPER(LABEL) LIKE UPPER("%arterial blood%")
ORDER BY label


""")
print(blood_pressure_df)


    itemid                                  label
0     8364                        ABP [Diastolic]
1        6                         ABP [Systolic]
2   225310                       ART BP Diastolic
3   225309                        ART BP Systolic
4   228151     Aortic Pressure Signal - Diastolic
5   228152      Aortic Pressure Signal - Systolic
6     8555             Arterial BP #2 [Diastolic]
7     6701              Arterial BP #2 [Systolic]
8     8368                Arterial BP [Diastolic]
9       51                 Arterial BP [Systolic]
10  220058   Arterial Blood Pressure Alarm - High
11  220056    Arterial Blood Pressure Alarm - Low
12  220051      Arterial Blood Pressure diastolic
13  220052           Arterial Blood Pressure mean
14  220050       Arterial Blood Pressure systolic
15    8502                    BP Cuff [Diastolic]
16    3313                     BP Cuff [Systolic]
17    8503                BP Left Arm [Diastolic]
18    3315                 BP Left Arm [Systolic]




#### 6. **Find the cohort who had Blood Pressure Measurements using these codes**

We extract data from *mimiciv_icu.chartevents* using the code found in last step with codes (220051,220050,220052,220056). We also tried other events, but only chartevents provide us valid dataset. This dataset is named as bp  table.


In [13]:
bp = run_query("""

SELECT *
FROM `physionet-data.mimiciv_icu.chartevents`  
WHERE itemid in (220051,220050,220052,220056)
ORDER BY subject_id
""")
print(bp.head())

   subject_id   hadm_id   stay_id           charttime           storetime  \
0    10002013  23581541  39060235 2160-05-19 06:00:00 2160-05-19 06:39:00   
1    10002013  23581541  39060235 2160-05-19 09:00:00 2160-05-19 09:18:00   
2    10002013  23581541  39060235 2160-05-18 22:10:00 2160-05-18 22:11:00   
3    10002013  23581541  39060235 2160-05-19 04:04:00 2160-05-19 05:33:00   
4    10002013  23581541  39060235 2160-05-18 21:00:00 2160-05-18 21:16:00   

   itemid value  valuenum valueuom  warning  
0  220050   136     136.0     mmHg        0  
1  220051    63      63.0     mmHg        0  
2  220051    53      53.0     mmHg        0  
3  220050    92      92.0     mmHg        0  
4  220050    97      97.0     mmHg        0  


#### 7.   **Data cleaning and combining**
We only keep the columns we focus on for both two datasets, the patient identidfiers and the event time.

We also replace the blood pressure measurement code with string, to clear indicates the measure events. Since we need to build a timeline data, so we need to convert the time data into datetime format to make sure they can be sorted later. 

Then we group data by stay_id, then we will can seperate each patient for each stay and check them one by one.

In [14]:
# Only Keep patient identifier and event end time, naming all intravenous vasoactive drugs as "drug"
clean_drugs = drugs[["subject_id","hadm_id",'stay_id', 'endtime']]
clean_drugs["event"] = "drug"
clean_drugs = clean_drugs.rename(columns={"endtime": "event_time"})

# Only Keep patient identifier and event end time, rename all blood pressure event
clean_bp = bp[["subject_id","hadm_id",'stay_id', 'charttime','itemid','valuenum']]
clean_bp['itemid'] = clean_bp['itemid'].replace(220051,"diastolic")
clean_bp['itemid'] = clean_bp['itemid'].replace(220052,"mean")
clean_bp['itemid'] = clean_bp['itemid'].replace(220050,"systolic")
clean_bp['itemid'] = clean_bp['itemid'].replace(220056,"alarm_low")

clean_bp = clean_bp.rename(columns={"charttime": "event_time","itemid":"event"})

In [15]:
# Combine all events and sort them by time

hypotension = pd.concat([clean_drugs,clean_bp])
hypotension['event_time']= pd.to_datetime(hypotension['event_time'])

hypotension = hypotension.sort_values(by = ["stay_id","event_time"])

In [16]:
# We only use ICU data so only consider stay id
# Groupby stay_id and check if each stay_id meet the definition of profound hypotension
gby_id = hypotension.groupby("stay_id")

#### 8. **Label state for each timeslot**

After grouping and ordering we want to provide label for each time slot.

We have three state labels **(Hypotension, Drug and Normal)**.

We first label the state with the Drug, if the patient takes the Drug in this timeslot, then this will be labelled as **Drug**. 
 
Then, we follow the rule to determine if the patient is having **Hypotension** right now.  

*   systolic blood pressure is <90 mmHg or
*   the mean arterial pressure is <65 mmHg or
*   Arterial Blood Pressure Alarm - Low

We drop the effect of diastolic blood pressure since it is hard for us to catch the drop in relative 

The rest of them will be labelled as **Normal**.

A filter is also apply to check if all three labels are present in each stay.



In [17]:
# Lable each record from each stay

labelled_dataset = []
for stay_id, stay_df in gby_id:

    label_set = set(stay_df["event"].tolist())

    # each patient should have three stage, Hypotension last > 60min, take drug and back to normal.
    # Thus we have to make sure this patient have taken drug or have blood pressure measurements
    if ("drug" in label_set) and (("systolic" or "alarm_low" or "mean") in label_set):
        #print(stay_df)

        # Here we use "absolute" to define Hypotension as well as the Blood Pressure Alarm low
        # If systolic blood pressure <90 mmHg or mean arterial pressure <65 mmHg or have Blood Pressure alarm Low, then it will be label as "Hypotension"
        # The drug taking event will be label as "Drug", the rest of event will be label as "Normal"
        label = []
        for i, row in stay_df.iterrows():
            if ((row["event"] == "systolic") and  (row["valuenum"] < 90)) or ((row["event"] == "mean") and  (row["valuenum"] < 45)   ) or (row["event"] == "alarm_low") :
                label.append("Hypotension")
            elif (row["event"] == "drug"):
                label.append("Drug")
            else:
                label.append("Normal")
        stay_df["label"] = label

        # Filter to check if all three stages are present
        if (len(set(stay_df["label"]))) == 3:
        
            labelled_dataset.append(stay_df)

#### 9. **Check if each patient for each stay is valid for check_profound_hypotension**
Profound Hypotension has three stages: **`(Hypotension last>60min -> Take Drugs -> Blood Pressure Corrected)`**. 


We will iterate each row(timeslot) of each stay. We will record the time if the label indicates Hypotension/Druge/Normal. We use Flags to trace the whole process to ensure all events happen in order. 

For example, if the process: `<Hypotension(last 30min), Normal(last 1min), Hypotension(last 40min), Drug(take after 30min), Normal>.` 

This will be seen as invalid since the Hypotension last time should be continuous, not separated.

The drug would be valid if it were taken within 90mins after the last Hypotension label was observed.

For example `<Hypotension(90min), Normal(5min), Drug(take after 10min), Normal> `will be seen as valid. But to ensure the drug is affected, we produce a constraint for drug-taken time: we need the drug to be taken within 90 mins.

For example, `<Hypotension(90min), Drug(take after 120min), Normal>` and `<Hypotension(last 90min), Normal(last 90min), Drug(take after 120min), Normal> ` will be not seen as valid. Since we can't determine if the drug works well.

We may need to constrain the drug effect time further since it may need some time for a drug to correct blood pressure. But we do not have the actual drug effect time. So we just simply see if there is a Normal label that occurs after Durg. If it has, it can be treated as valid.



In [18]:
# Design algorithm for check profound hypotension
# First we need to find where Hypotension start
# then we will record the Hypotension last time
# If the Hypotension last over 60 min then it means profound hypotension start and our first conditions is satisfied
# Then we will check if the patient take drugs with in 90 mins
# After Drug taking, if the patient is correct then it means the patient meet definition and we will break the iteration to save time

def check_profound_hypotension_2(df):
    hypo_start = False
    hypo_happening = False
    profound_start = False
    drug_take = False
    back_normal = False
    hypo_end = False
    
    for index, row in df.iterrows():
        if row["event"] == "Hypotension" and not hypo_start:
            hypo_start_time = row["event_time"]
            hypo_start = True
            #print("hypo start")

        elif row["event"] == "Hypotension" and hypo_start:
            hypo_keep_time = row["event_time"]
            hypo_last_time = (row["event_time"] - hypo_start_time).total_seconds() / 60
            hypo_happening = True
            hypo_end = False
            #print("hypo last",hypo_last_time)
            if hypo_last_time > 60:
                profound_start = True


        elif row["event"] == "Normal" and hypo_start and not hypo_end:
            hypo_end_time = row["event_time"]
            last_time = (hypo_end_time - hypo_start_time).total_seconds() / 60
            #print("hypo end", last_time)
            hypo_start = False
            if last_time > 60:
                profound_start = True
                hypo_end = True

        if profound_start and row["event"] == "Drug":
            drug_time = row["event_time"]
            if hypo_end:
                drug_after = (drug_time - hypo_end_time).total_seconds() / 60
            else:
                drug_after = (drug_time - hypo_keep_time).total_seconds() / 60
            #print("Profound and Take Drug",drug_after)
            if drug_after <= 90:
                drug_take = True
                #print("Vaild Drug")
                

        if drug_take and row["event"] == "Normal":
            normal_time = row["event_time"]
            drug_take_time = (normal_time - drug_time).total_seconds() / 60
            hypo_start = False
            profound_start = False
            drug_take = False
            hypo_happening = False
            #print("Back to Normal and Hypo end",drug_take_time)
            return True
    return False

#### 10. **Deal with conflict labels. Only keep one label for each timeslot**
For the timeslot have multiple events and the events conflicts, we will use the priority of **<Drug, Hypotension, Normal>** to label each timeslot

For example:

If a patient has to mean arterial and systolic blood pressure simultaneously, but only systolic blood pressure measurement indicates Hypotension, and the mean arterial pressure measurement does not. Follow the priority list, we will label this timeslot as **Hypotension**.

If a Drug and Hypotension occurs at the same time, we will keep the **Drug** label.



After that, we apply the check_profound_hypotension function to indicate if the current stay is valid for profound hypotension, if they are we will keep them with `profound_hypotension == True`, otherwise we will drop them.

In [19]:
# Check if each patient is valid for profound hypotension
# However, we found the patient usually have different blood pressure measure at the same time
# Some of the measurements indicates Hypotension, but some does not which bring conflicts
# For example, the patient have systolic blood pressure and mean arterial pressure at the same time, but only the systolic blood pressure satisfy the hypotension definition.
# To deal with this, we group events by their time, if there exist one measurement that indicates Hypotension in each timeslot, then we will label this timeslot as "Hypotension"
subject_id_list = []
hadm_id_list = []
stay_id_list = []

for stay in labelled_dataset:
    # Groupby eventtime
    gb_time = stay.groupby("event_time")

    event_list = []
    time_list = []

    # We only keep one label for each timeslot
    # If we take drug and have hypotension at the same timeslot, we will keep "Drug" since we always take durg when we are suffering hypotension.
    # If we have only one measurements indicates "Hypotension", the rest of measurement indicates "Normal", we would keep "Hypotension"
    for i,j in gb_time:
        condition_list = j["label"].tolist()
        time_list.append(j["event_time"].tolist()[0])
        if "Drug" in condition_list:
            event_list.append("Drug")
        elif "Hypotension" in condition_list:
            event_list.append("Hypotension")
        else:
            event_list.append("Normal")

    # After that each timeslot will only have one label        
    perday = pd.DataFrame({'subject_id':j["subject_id"].tolist()[0], 'hadm_id':j["hadm_id"].tolist()[0], 'stay_id':j["stay_id"].tolist()[0],
                    "event_time":time_list,"event":event_list})

    # Then we have a series of events that are in time order and we can use this timeseries data sequence to check profound hypotension
    # We use the check_profound_hypotension_2 to check if the patient meet the definition
    # if it does we will record the subject_id, hadm_id and stay_id for later data matching
    if(check_profound_hypotension_2(perday)):
        subject_id_list.append(perday["subject_id"][0])
        hadm_id_list.append(perday["hadm_id"][0])
        stay_id_list.append(perday["stay_id"][0])

cohort = pd.DataFrame({'subject_id':subject_id_list, 'hadm_id':hadm_id_list, 'stay_id':stay_id_list, "profound_hypotension":True})

In [20]:
cohort

,subject_id,hadm_id,stay_id,profound_hypotension
0,14569364,20870580,30005707,True
1,18756985,21715366,30006983,True
2,16828280,22369507,30007565,True
3,18885050,26109456,30008148,True
4,15993831,21791028,30009339,True
...,...,...,...,...
6415,15530538,21057134,39986775,True
6416,19962126,21472938,39986786,True
6417,17526143,29577504,39996073,True
6418,17874908,25979104,39998012,True


#### 11. **Extract patients' ICD Code**
A column that states whether Any ICD code for hypotension was added to their list of diagnoses when they were discharged
I95 is the Version 10 ICD code for Hypotension.

We use ***mimiciv_hosp.diagnoses_icd*** table to search all **icd_code** that like **("I95%")** since all hypotension related code starts with **I95**. 


We name this table the ICD table, which contains the patients' identifiers **hadm_id** with codes like **I95** and their ICD codes.


In [21]:
ICD = run_query("""

SELECT*  FROM `physionet-data.mimiciv_hosp.diagnoses_icd` 

WHERE icd_code LIKE ("I95%")

""")
print(ICD.head())

   subject_id   hadm_id  seq_num icd_code  icd_version
0    10011427  20219031       26    I9589           10
1    10012848  27928632       28     I951           10
2    10013310  22098926       32     I959           10
3    10014354  26228185       27     I959           10
4    10030753  23960805       26     I951           10


#### 12. **Link the ICD table and the Cohort Table**
We use **hadm_id** as the key, iterate each **hadm_id** from the Cohort table and check if this **hadm_id** occurs in the ICD table. 

Finally, a boolean value indicates if  this patient has been record as **Hypotension** through ICD code is added to the final cohort table.


In [22]:
result = []
for i in cohort["hadm_id"]:
    if i in ICD["hadm_id"]:
        result.append(True)
    else:
        result.append(False)

In [23]:
cohort["ICD_CODE"] = result

#### Final Cohort Table

In [24]:
cohort

,subject_id,hadm_id,stay_id,profound_hypotension,ICD_CODE
0,14569364,20870580,30005707,True,False
1,18756985,21715366,30006983,True,False
2,16828280,22369507,30007565,True,False
3,18885050,26109456,30008148,True,False
4,15993831,21791028,30009339,True,False
...,...,...,...,...,...
6415,15530538,21057134,39986775,True,False
6416,19962126,21472938,39986786,True,False
6417,17526143,29577504,39996073,True,False
6418,17874908,25979104,39998012,True,False


### References


*   https://www.fortherecordmag.com/archives/1013p10.shtml
*   https://ontoserver.csiro.au/shrimp/?concept=45007003&version=http%3A%2F%2Fsnomed.info%2Fsct%2F32506021000036107%2Fversion%2F20220831&valueset=http%3A%2F%2Fsnomed.info%2Fsct%2F32506021000036107%3Ffhir_vs
* https://icd.who.int/browse10/2019/en#/I95
